In [1]:
# GPU information
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jan  5 17:38:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Import Libraries

In [2]:
# import libraries
from matplotlib import pyplot as plt
import seaborn as sb

from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import os
#import random


#from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import Normalizer

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
#from keras import models
#from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
#from keras.layers import Activation, Dropout, Flatten, Dense
from keras.models import load_model

# Load the Dataset

In [3]:
#from os.path import join
from google.colab import drive
ROOT = "/content/drive"
drive.mount(ROOT)

# change directory to the database
%cd '/content/drive/MyDrive/Colab Notebooks/AML_22-23_SN19017126'

# list items
print('\nCurrent directory: \n')
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/AML_22-23_SN19017126

Current directory: 

A1  B1_CNN.h5			     B2_CNN.h5	   main.ipynb
A2  B1_face_shape_recognition.ipynb  Datasets	   __pycache__
B1  B2				     landmarks.py


In [4]:
# extracting the eye_color label from cartoon dataset
cartoon_label = pd.read_csv('./Datasets/cartoon_set/labels.csv', sep = '\t')
eye_label = cartoon_label.drop(columns = [cartoon_label.columns[0]]).drop(columns = [cartoon_label.columns[2]])
eye_label['eye_color'] = eye_label['eye_color'].apply(str)
print(eye_label)

y_train = eye_label
print('Shape of y_train: ', y_train.shape)

     eye_color file_name
0            1     0.png
1            2     1.png
2            2     2.png
3            2     3.png
4            0     4.png
...        ...       ...
9995         3  9995.png
9996         0  9996.png
9997         1  9997.png
9998         0  9998.png
9999         2  9999.png

[10000 rows x 2 columns]
Shape of y_train:  (10000, 2)


In [5]:
# the link of the image 
cartoon_img = ('./Datasets/cartoon_set/img')

In [6]:
# create data argumentation
data_argumantation = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255.,
    validation_split = 0.25,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,
    #width_shift_range=[-100, +100],
    #height_shift_range=[-100, +100],
    brightness_range=[0.5, 1.5],
    #zoom_range=[0.5, 1.5]
)

In [11]:
# Generata batches of image dataset and labels for training
print("Training Dataset: ")
train_generator = data_argumantation.flow_from_dataframe(
    
        dataframe = y_train, 
        directory = cartoon_img,  #path to read images
        x_col = "file_name", 
        y_col = "eye_color",
        class_mode = 'categorical',  #must include the class of each image
        target_size = (128, 128),  #(32,32),
        batch_size = 128, 
        subset = 'training') 
   
#  Generata batches of image dataset and labels for validation 
print("\n")
print("Validation Dataset: ")
validation_generator = data_argumantation.flow_from_dataframe(
        dataframe = y_train, 
        directory = cartoon_img,  #path to read images
        x_col = "file_name", 
        y_col = "eye_color",
        class_mode = 'categorical',  #must include the class of each image
        target_size = (128, 128),  #(32,32),
        batch_size = 128, 
        subset = 'validation')

Training Dataset: 
Found 7500 validated image filenames belonging to 5 classes.


Validation Dataset: 
Found 2500 validated image filenames belonging to 5 classes.


# CNN

In [12]:
# reset the model
keras.backend.clear_session()

# starting point 
cnn_model= Sequential()

# Block 1
# Add a convolutional layer with 
cnn_model.add(Conv2D(filters=16, kernel_size=3, padding="same", activation="relu", input_shape=(128,128,3)))
# Add a max pooling layer with 
cnn_model.add(MaxPooling2D(pool_size=2, padding='same'))

# Block 2
# Add a convolutional layer with 
cnn_model.add(Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
# Add a max pooling layer with 
cnn_model.add(MaxPooling2D(pool_size=2, padding='same'))

# Block 3
# Add a convolutional layer with 
cnn_model.add(Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
# Add a max pooling layer with 
cnn_model.add(MaxPooling2D(pool_size=2, padding='same'))

# Block 4
# Add a convolutional layer with 
cnn_model.add(Conv2D(filters=128, kernel_size=3, padding="same", activation="relu"))
# Add a max pooling layer with 
cnn_model.add(MaxPooling2D(pool_size=2, padding='same'))

# Add a dropout layer with a rate of 0.5
cnn_model.add(Dropout(0.5))

# Flatten the output of the convolutional layers into a single vector
cnn_model.add(Flatten())

# Add a fully connected layer with 128 units and a ReLU activation function
cnn_model.add(Dense(128, activation='relu'))

# Add a final fully connected layer with a softmax activation function for classification
num_classes = 5
cnn_model.add(Dense(num_classes, activation='softmax'))

# Compile the model with a categorical cross-entropy loss function and Adam optimization
cnn_model.compile(loss=keras.losses.categorical_crossentropy, 
                  optimizer='adam', 
                  metrics=['accuracy'])


In [13]:
# Print the summary of the model and the no. of trainable/non-trainable parameters
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 64)       0

In [14]:
training_history = cnn_model.fit(
                                train_generator,
                                epochs=20,
                                steps_per_epoch=train_generator.samples // train_generator.batch_size, 
                                validation_data=validation_generator,
                                validation_steps=validation_generator.samples // validation_generator.batch_size
                                )


Epoch 1/20
58/58 [==============================] - 94s 2s/step - loss: 1.6212 - accuracy: 0.1995 - val_loss: 1.6097 - val_accuracy: 0.1945
Epoch 2/20
58/58 [==============================] - 91s 2s/step - loss: 1.6088 - accuracy: 0.2098 - val_loss: 1.5962 - val_accuracy: 0.2220
Epoch 3/20
58/58 [==============================] - 91s 2s/step - loss: 1.3317 - accuracy: 0.4015 - val_loss: 1.0902 - val_accuracy: 0.5222
Epoch 4/20
58/58 [==============================] - 91s 2s/step - loss: 0.9797 - accuracy: 0.5844 - val_loss: 0.8907 - val_accuracy: 0.6221
Epoch 5/20
58/58 [==============================] - 91s 2s/step - loss: 0.8372 - accuracy: 0.6276 - val_loss: 0.7302 - val_accuracy: 0.6608
Epoch 6/20
58/58 [==============================] - 91s 2s/step - loss: 0.7199 - accuracy: 0.6595 - val_loss: 0.6681 - val_accuracy: 0.6793
Epoch 7/20
58/58 [==============================] - 91s 2s/step - loss: 0.6823 - accuracy: 0.6742 - val_loss: 0.6450 - val_accuracy: 0.7007
Epoch 8/20
58/58 [==

# Save the CNN model

In [15]:
B2_CNN = 'B2_CNN.h5'

cnn_model.save(B2_CNN)

# Evaludate on the Test Dataset

In [16]:
# extracting the test dataset
cartoon_test_label = pd.read_csv('./Datasets/cartoon_set_test/labels.csv', sep = '\t')
eye_test_label = cartoon_test_label.drop(columns = [cartoon_test_label.columns[0]]).drop(columns = [cartoon_test_label.columns[2]])
eye_test_label['eye_color'] = eye_test_label['eye_color'].apply(str)
print(eye_test_label)

     eye_color file_name
0            2     0.png
1            1     1.png
2            0     2.png
3            0     3.png
4            3     4.png
...        ...       ...
2495         4  2495.png
2496         2  2496.png
2497         2  2497.png
2498         4  2498.png
2499         3  2499.png

[2500 rows x 2 columns]


In [17]:
# image path for new test images
cartoon_test_img = ('./Datasets/cartoon_set_test/img')

In [18]:
test_generator = data_argumantation.flow_from_dataframe(
        dataframe = eye_test_label, 
        directory = cartoon_test_img,
        x_col = "file_name", 
        y_col = "eye_color",
        batch_size=1,
        class_mode='categorical', 
        target_size=(128,128),
        shuffle=False)
test_steps = test_generator.samples
print(test_steps)

Found 2500 validated image filenames belonging to 5 classes.
2500


In [19]:
# testing model on new dataset
B2_model = load_model(B2_CNN)

In [20]:
# printing training loss and accuracy
train_score = B2_model.evaluate(train_generator)

print('Train loss: '+ str(train_score[0]))
print('Train Accuracy: '+ str(train_score[1]))

59/59 [==============================] - 69s 1s/step - loss: 0.3498 - accuracy: 0.8319
Train loss: 0.349770724773407
Train Accuracy: 0.8318666815757751


In [21]:
# printing validation loss and accuracy
valid_score = B2_model.evaluate(validation_generator)
                                          
print('Validation loss: '+ str(valid_score[0]))
print('Validation Accuracy: '+ str(valid_score[1]))

20/20 [==============================] - 23s 1s/step - loss: 0.3666 - accuracy: 0.8188
Validation loss: 0.3665755093097687
Validation Accuracy: 0.8187999725341797


In [22]:
test_generator.reset()
B2_pred = B2_model.predict(test_generator)

# determine the maximum activation value for each sample
B2_pred_cnn = np.argmax(B2_pred, axis=1)

# label each predicted value to correct face shape
labels = (test_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in B2_pred_cnn]

2500/2500 [==============================] - 485s 194ms/step


In [23]:
accuracy_cnn = accuracy_score(test_generator.classes, B2_pred_cnn)

In [24]:
print(classification_report(test_generator.classes, B2_pred_cnn ))
print("Accuracy of CNN model: {:.2f}%".format(accuracy_cnn*100))

              precision    recall  f1-score   support

           0       0.98      0.76      0.86       506
           1       0.92      0.80      0.85       483
           2       0.61      0.96      0.75       525
           3       0.89      0.81      0.85       514
           4       0.97      0.82      0.89       472

    accuracy                           0.83      2500
   macro avg       0.88      0.83      0.84      2500
weighted avg       0.87      0.83      0.84      2500

Accuracy of CNN model: 83.16%
